<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/make_super_extend_dataset_for_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Create extended dataset for YOLOv5 implementation**

#Olympia
・For_validation ：treated 169枚、control 169枚 ここからテスト用に残すもの→treated 69枚、control 69枚 controlのうち100枚→train/valに回す

・Treated 640px：treated 561枚 テスト用に残した69枚以外もの全て（grav:492枚）をtrain/valに回す

#Handai
・GO_extended_dataset

control: 1886枚

treatable: 1879枚

#合計：
treatable: 1879 + 492

control: 1886 + 100

testset: Olympia treated 69, control 69

##既存のYOLOでbounding boxを作成 --> classは既知のもので。Glaucomaのrepositoryを参考にしてYOLO datasetを作成。

In [3]:
import pandas as pd
import os
import glob
import shutil
import glob
import random
from PIL import Image
import time
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2


# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dst_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO"

if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir)
os.makedirs(f"{dst_dir}/train_val")
os.makedirs(f"{dst_dir}/train_val/grav")
os.makedirs(f"{dst_dir}/train_val/cont")
os.makedirs(f"{dst_dir}/test")
os.makedirs(f"{dst_dir}/test/grav")
os.makedirs(f"{dst_dir}/test/cont")

In [9]:
################
# olympia dataset
################

import os
import shutil
import random

random_state = 1

# Control images
src_cont = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/for_validation/control_640px"
test_cont = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/cont"
trainval_cont = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

cont_images = os.listdir(src_cont)
random.Random(random_state).shuffle(cont_images)

test_cont_images = cont_images[:69]
trainval_cont_images = cont_images[69:]

for image in test_cont_images:
    src_path = os.path.join(src_cont, image)
    dst_path = os.path.join(test_cont, image)
    shutil.move(src_path, dst_path)

for image in trainval_cont_images:
    src_path = os.path.join(src_cont, image)
    dst_path = os.path.join(trainval_cont, image)
    shutil.move(src_path, dst_path)


# Grav images
src_grav = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/for_validation/treated_640px"
test_grav = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/grav"
trainval_grav = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"

grav_images = os.listdir(src_grav)
random.Random(random_state).shuffle(grav_images)

test_grav_images = grav_images[:69]
trainval_grav_images = grav_images[69:]

for image in test_grav_images:
    src_path = os.path.join(src_grav, image)
    dst_path = os.path.join(test_grav, image)
    shutil.move(src_path, dst_path)

for image in trainval_grav_images:
    src_path = os.path.join(src_grav, image)
    dst_path = os.path.join(trainval_grav, image)
    shutil.move(src_path, dst_path)
